In [7]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction import text  
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import sqlite3
cnx = sqlite3.connect('db.sqlite3')
df = pd.read_sql_query("SELECT * FROM application_data_set",cnx)

def build_train_test():
    
    # Create your connection.
    
    #data = pd.read_csv('spam.csv') 
    #data = data[['Label','EmailText']]
    #data.dropna(axis='index',how='any',inplace=True)
    
    train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

    return train_set['EmailText'],train_set['Label'],test_set['EmailText'],test_set['Label']

In [2]:

X_train,Y_train,X_test,Y_test = build_train_test()
vectorizer = text.CountVectorizer(stop_words="english")
counts = vectorizer.fit_transform(X_train.values)
spam_nb = MultinomialNB()
spam_nb.fit(counts,Y_train.values)

MultinomialNB()

In [3]:
def model_accuracy():
    examples = vectorizer.transform(X_test.values)
    predicted = spam_nb.predict(examples)
    prediction_bool = pd.Series(predicted==Y_test.values)
    true_count = prediction_bool[prediction_bool==True].count()
    false_count = prediction_bool[prediction_bool==False].count()
    print((100*true_count)/(true_count+false_count))

In [4]:
def test_model(EmailText):
    test_vect = vectorizer.transform(np.array([EmailText]))
    predict_test_vect = spam_nb.predict(test_vect)
    return predict_test_vect[0]

In [5]:
model_accuracy()

98.74776386404294


In [8]:
df.to_sql(name='testdb',con=cnx)